<a href="https://colab.research.google.com/github/kuds/rl-drone/blob/main/RL%20Drone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title # Reinforcement Learning Drone Control in MuJoCo
#@markdown This script will install the necessary libraries, define a drone environment,
#@markdown and train a PPO agent to fly the drone to a target.

#@markdown ## 1. Install Dependencies
#@markdown This cell installs `mujoco`, `gymnasium` for creating the environment,
#@markdown and `stable-baselines3` for the reinforcement learning algorithm.
!pip install mujoco
!pip install gymnasium
!pip install stable-baselines3

#@markdown ## 2. Imports
#@markdown Import the necessary libraries.
import gymnasium as gym
from gymnasium import spaces
import mujoco
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

#@markdown ## 3. Define the Drone XML Model
#@markdown This is the physical definition of our drone in MuJoCo's XML format.
drone_xml = """
<mujoco model="drone">
  <compiler angle="degree" coordinate="local" inertiafromgeom="true"/>
  <option integrator="RK4" timestep="0.01"/>
  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <body name="drone" pos="0 0 1">
      <camera name="track" mode="trackcom" pos="0 -3 1" xyaxes="1 0 0 0 0 1"/>
      <geom type="box" size=".1 .1 .02" rgba=".9 .9 .9 1" mass="1"/>
      <joint type="free" limited="false"/>
      <!-- Rotors -->
      <body name="rotor1" pos="0.1 0.1 0">
        <joint type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
      <body name="rotor2" pos="-0.1 0.1 0">
        <joint type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor3" pos="0.1 -0.1 0">
        <joint type="hinge" axis="0 0 -1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="1 0 0 1"/>
      </body>
      <body name="rotor4" pos="-0.1 -0.1 0">
        <joint type="hinge" axis="0 0 1"/>
        <geom type="cylinder" size="0.05 0.01" rgba="0 1 0 1"/>
      </body>
    </body>
    <!-- Target -->
    <body name="target" pos="1 1 2">
        <geom type="sphere" size="0.1" rgba="1 0 0 0.5" />
    </body>
  </worldbody>
  <actuator>
    <motor joint="rotor1" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor2" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor3" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
    <motor joint="rotor4" gear="0.1" ctrllimited="true" ctrlrange="0 1000"/>
  </actuator>
</mujoco>
"""

#@markdown ## 4. Create the Custom Gym Environment
class DroneEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(DroneEnv, self).__init__()

        # Load the drone model
        self.model = mujoco.MjModel.from_xml_string(drone_xml)
        self.data = mujoco.MjData(self.model)

        # Define action and observation space
        # Actions are the 4 rotor thrusts
        self.action_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)

        # Observations: drone_pos(3), drone_vel(3), drone_orient(4), target_pos(3)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(13,), dtype=np.float32)

        # Target position
        self.target_position = np.array([1.0, 1.0, 2.0])

    def step(self, action):
        # Apply action
        self.data.ctrl[:] = action * 1000  # Scale action to actuator range
        mujoco.mj_step(self.model, self.data)

        # Get observation
        obs = self._get_obs()

        # Calculate reward
        drone_pos = self.data.qpos[0:3]
        dist_to_target = np.linalg.norm(drone_pos - self.target_position)

        # Reward for being close to the target
        reward = -dist_to_target

        # Penalty for being tilted
        orientation = self.data.qpos[3:7]
        # A simple measure of tilt: z-component of the drone's up vector
        # For a quaternion [w, x, y, z], the z-axis of the rotated frame is
        # [2*(xz + wy), 2*(yz - wx), 1 - 2*(x^2 + y^2)]
        up_z = 1 - 2 * (orientation[1]**2 + orientation[2]**2)
        reward -= (1 - up_z) * 0.5

        # Penalty for control effort
        reward -= np.sum(action) * 0.01

        # Check if done
        done = False
        if dist_to_target < 0.2:
            reward += 100  # Bonus for reaching the target
            done = True
        if drone_pos[2] < 0.1: # Crashed
            reward -= 100
            done = True

        return obs, reward, done, False, {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        # Reset the simulation
        mujoco.mj_resetData(self.model, self.data)

        # Randomize target position
        self.target_position = np.random.rand(3) * 2 + np.array([-1, -1, 1])
        self.model.body('target').pos[:] = self.target_position

        return self._get_obs(), {}

    def _get_obs(self):
        return np.concatenate([
            self.data.qpos[0:3],    # Drone position
            self.data.qvel[0:3],    # Drone linear velocity
            self.data.qpos[3:7],    # Drone orientation (quaternion)
            self.target_position
        ])

    def render(self, mode='human'):
        # This is a placeholder for rendering.
        # For a full visualization, you would need to integrate a renderer
        # like mujoco.viewer.
        pass

#@markdown ## 5. Train the Reinforcement Learning Agent
#@markdown We will use the PPO algorithm from Stable Baselines3.

# Create a vectorized environment
env = DummyVecEnv([lambda: DroneEnv()])

# Instantiate the PPO model
model = PPO("MlpPolicy", env, verbose=1)

# Train the model
#@markdown You can adjust the number of timesteps for training.
#@markdown More timesteps will take longer but may result in a better policy.
training_timesteps = 10000 #@param {type:"integer"}
model.learn(total_timesteps=training_timesteps)

# Save the model
model.save("ppo_drone")

print("\nTraining finished and model saved as 'ppo_drone.zip'")

#@markdown ## 6. Evaluate the Trained Agent
#@markdown Now, let's see how our trained drone performs.

# Load the trained model
model = PPO.load("ppo_drone")

# Create a new environment for evaluation
eval_env = DroneEnv()
obs, _ = eval_env.reset()

print("\nStarting evaluation...")
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, _, info = eval_env.step(action)
    if dones:
        print(f"Episode finished after {i+1} timesteps.")
        obs, _ = eval_env.reset()

print("Evaluation finished.")

#@markdown ### To visualize the drone, you would typically use `mujoco.viewer`,
#@markdown but this is not directly supported in Colab's output cells.
#@markdown To see the drone fly, you would run this script in a local
#@markdown environment with a display.

ValueError: Error: unknown transmission target 'rotor1' for actuator id = 0
Element name '', id 0, line 35